In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
from rsnn.signals.spike_train import SpikeTrain, PeriodicSpikeTrain

/Users/haguettaz/Documents/RSNN/src/rsnn/signals/spike_train.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [18]:
[3, 4]/3

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [3]:
spike_train = PeriodicSpikeTrain(10)

In [16]:
period = 100
nominal_rate = 0.1
abs_refractory_time = 5
rel_refractory_time = 10
res = 1e-3

In [17]:
spike_train.random(period, nominal_rate, abs_refractory_time, rel_refractory_time, res)

[ 4.51282630e-04  1.24340023e-02  9.80887421e-02  2.87663399e-01
  3.52508171e-01  1.93296291e-01  4.92877388e-02  5.92809211e-03
  3.33673959e-04  8.51820240e-06  8.87312750e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


sampling:   0%|          | 0/10 [00:00<?, ?it/s]

[ 9.58025389 38.20925389 59.97925389 71.98825389 89.69825389]
[28.61472377 40.70772377 59.12672377 89.37172377]
[16.91637066 29.87737066 54.59037066 70.80637066 86.32937066 97.09137066]
[26.52777848 33.57677848 45.06177848 55.52677848 80.98477848 91.28677848]
[25.64391692 50.94291692 57.36291692 78.39691692 88.10091692]
[59.94764833 67.86164833]
[ 7.92236795 16.48836795 34.94836795 89.73436795]
[10.47876375 40.73076375 54.41476375 69.10676375 98.41876375]
[17.97996811 38.20496811 47.38796811 60.49396811 94.01196811]
[37.76117656 60.71517656 89.96817656]


In [ ]:
spike_train.num_spikes() / 100

In [2]:
random.seed(42)

In [9]:
T = 10
t1 = 2
t2 = 9

In [10]:
np.minimum(np.abs(t1 - t2), np.abs(T - np.abs(t1 - t2))) < eps

3

In [ ]:
# T -> 2 * np.pi
# t / T

In [4]:
rng = np.random.default_rng()

In [5]:
g = lambda t_: np.exp(- t_)

In [29]:
def argsort(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)

In [30]:
firing_times = np.concatenate(spike_train.firing_times).tolist()
sources = [neuron for _ in range(spike_train.num_spikes(neuron.idx)) for neuron in network.neurons]
indices = argsort(firing_times)

Phi = np.identity(N)
A = np.zeros((N, N))
A[1:,:-1] = np.identity(N-1)
for m in range(M):
    # current spike is spikes[N + m]
    for n in range(N):
        A[0, n] = sum(
            [
                input.weight * input.resp_deriv((firing_times[m] - input.delay - firing_times[indices[(m - n - 1) % M]]) % spike_train.duration) for input in sources[indices[m]].inputs if input.source is sources[indices[(m - n - 1) % M]] 
            ]
        )
    A[0] /= np.sum(A[0])
    Phi = A@Phi

NameError: name 'spike_train' is not defined

In [7]:
class SpikeTrain:
    def __init__(self, times, sources):
        if isinstance(sources, list):
            self.spikes = [Spike(t, src) for t, src in zip(times, sources)]
        else:
            self.spikes = [Spike(t, sources) for t in times]
                
    def __call__(self, ):
        return 

In [ ]:
# class RefractoryConnection:
#     def __init__(self, source, weight, filter, filter_deriv):
#         self.source = source
#         self.weight = weight
#         self.somatic_decay = somatic_decay
#     
#     def response(self, t):
#         tau = t - sources.firing_times
#         pos_tau = tau[tau > 0]
#         return - self.weight * np.sum((np.exp(- pos_tau / self.somatic_decay)) # fun could be anything
#     
#     # def response_derivative(self, t):
#     #     if t <= self.delay:
#     #         return 0
#     #     
#     #     return self.weight * ... # could be anything

class Connection:
    def __init__(self, source, delay, weight, resp, resp_deriv):
        self.source = source
        self.weight = weight
        self.delay = delay
        self.resp = resp
        self.resp_deriv = resp_deriv
            
    def signal(self, t):
        tau = t - self.delay - sources.firing_times
        pos_tau = tau[tau > 0]
        return self.weight * np.sum((self.resp(pos_tau)) 
                                    
    def signal_deriv(self, t):
        tau = t - self.delay - sources.firing_times
        pos_tau = tau[tau > 0]
        return self.weight * np.sum((self.resp_deriv(pos_tau)) 
                                    
                                    
    # def response_derivative(self, t):
    #     if t <= self.delay:
    #         return 0
    #     
    #     return self.weight * ... # could be anything

In [24]:
class Neuron:
    def __init__(self, firing_threshold):
        self.firing_threshold = firing_threshold
        self.inputs = []
        self.firing_times = np.array([])
    
    def potential(self, t):
        return sum([input.signal(t) for input in self.inputs])
    
    def potential_deriv(self, t):
        return sum([input.signal_deriv(t) for input in self.inputs])
    
    def step(self, t)

In [ ]:
import pickle 

In [23]:
class Network:
    # a set of neurons and inputs between them
    def __init__(self, num_neurons, firing_threshold):
        self.num_neurons = num_neurons
        self.neurons = [Neuron(firing_threshold) for l in range(self.num_neurons)]    
        self.firing_times = np.array([])
        self.firing_threshold = firing_threshold
        
    def rand(self, num_synapses, soma_decay, syn_decay, syn_delays_lim, syn_weights_lim, syn_weight_res, ref_weights_lim, ref_weight_res, seed=0):
        self.num_synapses = num_synapses 
        self.soma_decay = soma_decay
        self.syn_decay = syn_decay
        self.syn_delays_lim = syn_delays_lim
        self.syn_weights_lim = syn_weights_lim 
        self.syn_weight_res = syn_weight_res
        self.ref_weights_lim = ref_weights_lim
        self.ref_weight_res = ref_weight_res
        
        if soma_decay == syn_decay:
            syn_resp = lambda t_: t_ / soma_decay * np.exp(1 - t_ / self.soma_decay)
            syn_resp_deriv = lambda t_: np.exp(1 - t_ / soma_decay) * (1 - t_ / soma_decay) / soma_decay
        else:
            tmax = (math.log(syn_decay) - math.log(soma_decay)) / (1 / soma_decay - 1 / syn_decay)
            gamma = 1 / (math.exp(-tmax / soma_decay) - math.exp(-tmax / syn_decay))
            syn_resp = lambda t_: gamma * (np.exp(- t_ / soma_decay) - np.exp(- t_ / syn_decay))
            syn_resp_deriv = lambda t_: gamma * (np.exp(- t_ / syn_decay) / syn_decay - np.exp(- t_ / soma_decay) / soma_decay)
            
        ref_resp = lambda t_: np.exp(- t / soma_decay)
        ref_resp_deriv = lambda t_: - np.exp(- t / soma_decay) / soma_decay    
            
        random.seed(seed)
        for neuron in self.neurons:
            neuron.inputs = [
                Connection( 
                    self.neurons[random.randint(self.num_neurons)], 
                    random.uniform(*syn_delays_lim), 
                    random.uniform(*syn_weights_lim) if syn_weight_res is None else random.randrange(syn_weights_lim[0], syn_weights_lim[1] + syn_weight_res, syn_weight_res),
                    syn_resp,
                    syn_resp_deriv
                )
                for _ in range(num_synapses)
            ]
            neuron.inputs.append(
                Connection(
                    neuron, 
                    0,
                    random.uniform(*ref_weights_lim) if ref_weight_res is None else random.randrange(ref_weights_lim[0], ref_weights_lim[1] + ref_weight_res, ref_weight_res),
                    ref_resp,
                    ref_resp_deriv
                )
            )
            
    def run(self, t0, duration, dt, std_theta, free_ids):
        free_neurons = self.network[free_ids]
        
        for neuron in free_neurons:
            neuron.noisy_firing_threshold = neuron.firing_threshold + random.gauss(0, std_theta)
            
        for t in tqdm(np.arange(t0, t0 + duration, dt), desc="network simulation"):
            for neuron in free_neurons:
                neuron.step(t, dt, std_theta)        
        
    
    def memorize(self, spike_trains, max_level, min_slope, eps, res):
        summmaries = []
        for neuron in tqdm(self.neurons, desc="network optimization"):
            summmaries.append(neuron.memorize(spike_trains, max_level, min_slope, eps, res))
        return summmaries
    
        
    def load(self, dirname):
        with open(os.path.join(dirname, "network.pkl", "rb")) as f:
            config = pickle.load(f)
                  
        if self.num_neurons != config["num_neurons"]:
            raise ValueError(f"number of neuron does not match")
            
        self.firing_threshold = config["firing_threshold"]
        self.num_synapses = config["num_synapses"]
        self.soma_decay = config["soma_decay"]
        self.synapse_decay = config["synapse_decay"]
        self.synapse_delay_lim = config["synapse_delay_lim"]
        self.synapse_weight_lim = config["synapse_weight_lim"]
        self.synapse_weight_res = config["synapse_weight_res"]
        self.refractory_weights_lim = config["refractory_weight_lim"]
        self.refractory_weight_res = config["refractory_weight_res"]
        
        source_ids = config["source_ids"]
        delays = config["delays"]
        weights = config["weights"]
        
        if soma_decay == synapse_decay:
            synapse_resp = lambda t_: t_ / soma_decay * np.exp(1 - t_ / self.soma_decay)
            synapse_resp_deriv = lambda t_: np.exp(1 - t_ / soma_decay) * (1 - t_ / soma_decay) / soma_decay
        else:
            tmax = (math.log(synapse_decay) - math.log(soma_decay)) / (1 / soma_decay - 1 / synapse_decay)
            gamma = 1 / (math.exp(-tmax / soma_decay) - math.exp(-tmax / synapse_decay))
            synapse_resp = lambda t_: gamma * (np.exp(- t_ / soma_decay) - np.exp(- t_ / synapse_decay))
            synapse_resp_deriv = lambda t_: gamma * (np.exp(- t_ / synapse_decay) / synapse_decay - np.exp(- t_ / soma_decay) / soma_decay)
            
        refractory_resp = lambda t_: np.exp(- t / soma_decay)
        refractory_resp_deriv = lambda t_: - np.exp(- t / soma_decay) / soma_decay
        
        for neuron in self.neurons:
            neuron.inputs = [
                Connection( 
                    self.neurons[source_id], 
                    delay, 
                    weight,
                    synapse_resp,
                    synapse_resp_deriv
                )
                for source_id, delay, weight in zip(config["source_ids"][neuron.idx], config["delays"][neuron.idx], config["weights"][neuron.idx])
            ]
            neuron.inputs[-1].resp = refractory_resp
            neuron.inputs[-1].resp_deriv = refractory_resp_deriv
            neuron.firing_times = np.array(config["firing_times"][neuron.idx])
        
    def save(self, dirname):
        os.makedirs(dirname, exist_ok=True)
        
        # save network config as dict
        config = {
            "num_neurons": self.num_neurons, 
            "firing_threshold": self.firing_threshold,
            "num_synapses": self.num_synapses, 
            "soma_decay": self.soma_decay,
            "synapse_decay": self.synapse_decay,
            "synapse_delay_lim": self.synapse_delays_lim,
            "synapse_weight_lim": self.synapse_weights_lim,
            "synapse_weight_res": self.synapse_weight_res,
            "refractory_weight_lim": self.refractory_weights_lim,
            "refractory_weight_res": self.refractory_weight_res,
            "sources_ids": [[input.source.idx for input in neuron.inputs] for neuron in network.neurons],
            "delays": [[input.delay for input in neuron.inputs] for neuron in network.neurons],
            "weights": [[input.weight for input in neuron.inputs] for neuron in network.neurons],
            "firing_times": [neuron.firing_times.tolist() for neuron in network.neurons]
        }
        with open(os.path.join(dirname, "network.pkl", "wb") as f:
            pickle.dump(config, f)

In [13]:
w = rng.normal(0, 1, (1000,))

In [16]:
%%timeit
w = rng.normal(0, 1, (2000,))
y = np.array([g(rng.normal(0,1)) for k in range(2000)])
z = np.inner(w, y)

5.15 ms ± 23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
w = rng.normal(0, 1, (2000,))
z = 0
for k in range(2000):
    z += w[k]* g(rng.normal(0,1))

5.9 ms ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
w = rng.normal(0, 1, (2000,))
z += np.w[k]* g(rng.normal(0,1))